# Third-party models


In this notebook, we show how to integrate "external" models into seqme.

In [ ]:
from functools import partial

import numpy as np

import seqme as sm

Some models of interest are not available through e.g., PyPI or Huggingface - only the git repository may be available. Here we show how to run such models in seqme.


An external model is compatible with seqme if it satisfies the following three requirements:

- Repository is accessible using `git clone`, e.g. a public repository.
- Repository dependencies are installable using `pip install .`, e.g., by setup.py or pyproject.toml.
- Contains a function with signature `Callable[[list[str], ...], Any]` where the first parameter is called `sequences`.


Let's use a toy model in a github repository satisfying all three requirements. To do so, we need to define the function entry point, repository url and save directory.


In [ ]:
thirdparty_model = sm.models.ThirdPartyModel(
    entry_point="seqmetp.model:embed",
    repo_path="../plugins/thirdparty/main",
    # Path to an enviroments python executable.
    # If none, a venv is created using the current python executable.
    python_bin=None,
    repo_url="https://github.com/szczurek-lab/seqme-thirdparty",
    branch="main",  # Defaults: whole repository
)

`ThirdPartyModel` clones the model repository, creates a virtual enviroment (venv) (if `python_bin=None`) and installs the dependencies using `pip install .`.

Assuming everything went well, let's now compute a metric using this embedding model.


In [ ]:
def embedder(seq: list[str]) -> np.ndarray:
    return thirdparty_model(seq, batch_size=32)


embedder(["MKQW", "RKSPL"])

array([[44.,  8., 52., 32.],
       [55., 10., 65., 40.]])

In [ ]:
sequences = {
    "HydrAMP": ["MMRK", "RKSPL", "RRLSK", "RRLSK"],
    "hyformer": ["MKQW", "RKSPL"],
    "Random": ["KKKKK", "PLQ", "RKSPL"],
}

metrics = [sm.metrics.FBD(reference=sequences["Random"], embedder=embedder)]
df = sm.evaluate(sequences, metrics)

sm.show(df)

100%|██████████| 3/3 [00:00<00:00,  5.28it/s, data=Random, metric=FBD]  


,FBD↓
HydrAMP,215.60
hyformer,81.67
Random,0.00


## AMPlify


Let's also use AMPlify which is an antimicrobial peptide (AMP) classifier, i.e., outputs the probability a peptide has antimicrobial properties.

First you need an enviroment with python 3.9 (requirement of AMPlify), e.g., using conda:

In [ ]:
# !conda create -n amplify_env python=3.9 -y

Let's setup the model.

In [ ]:
thirdparty_model = sm.models.ThirdPartyModel(
    entry_point="predict:predict",
    repo_path="../plugins/thirdparty/amplify",
    # Set path to the python3.9 executable. If your default python version is 3.9,
    # then 'python_bin=None' will work out of the box.
    python_bin="/opt/anaconda3/envs/amplify_env/bin/python",
    repo_url="https://github.com/szczurek-lab/seqme-thirdparty",
    branch="amplify",
)

Assuming everything went well, let's now compute a metric using this predictive model.


In [ ]:
thirdparty_model(["MKQW", "RKSPL"], model_type="imbalanced", batch_size=128, n_ensembles=2)

array([0.00635906, 0.49806994], dtype=float32)

In [ ]:
sequences = {
    "HydrAMP": ["MMRK", "RKSPL", "RRLSK", "RRLSK"],
    "hyformer": ["MKQW", "RKSPL"],
    "Random": ["KKKKK", "PLQ", "RKSPL"],
}

discriminator = partial(thirdparty_model, model_type="balanced", n_ensembles=5, batch_size=128)

metrics = [sm.metrics.ID(predictor=discriminator, name="p_AMP (AMPlify)", objective="maximize")]
df = sm.evaluate(sequences, metrics)

sm.show(df)

100%|██████████| 3/3 [01:00<00:00, 20.03s/it, data=Random, metric=p_AMP (AMPlify)]  


,p_AMP (AMPlify)↑
HydrAMP,0.26±0.17
hyformer,0.24±0.24
Random,0.40±0.30


## amPEPpy


Let's also use amPEPpy which is an antimicrobial peptide (AMP) classifier, i.e., outputs the probability a peptide has antimicrobial properties.

First you need an enviroment with python 3.8, e.g., using conda:

In [ ]:
# !conda create -n ampeppy_env python=3.8 -y

Let's setup the model.

In [ ]:
thirdparty_model = sm.models.ThirdPartyModel(
    entry_point="amPEPpy.predict:predict",
    repo_path="./plugins/thirdparty/ampeppy",
    repo_url="https://github.com/szczurek-lab/seqme-thirdparty",
    branch="ampeppy",
    python_bin="/opt/anaconda3/envs/ampeppy_env/bin/python",
)

Assuming everything went well, let's now compute a metric using this predictive model.


In [ ]:
thirdparty_model(["MKQW", "RKSPL"])

array([0.49427083, 0.28333333])

In [ ]:
sequences = {
    "HydrAMP": ["MMRK", "RKSPL", "RRLSK", "RRLSK"],
    "hyformer": ["MKQW", "RKSPL"],
    "Random": ["KKKKK", "PLQ", "RKSPL"],
}

discriminator = thirdparty_model

metrics = [sm.metrics.ID(predictor=thirdparty_model, name="p_AMP (amPEPpy)", objective="maximize")]
df = sm.evaluate(sequences, metrics)

sm.show(df)

100%|██████████| 3/3 [00:10<00:00,  3.45s/it, data=Random, metric=p_AMP (amPEPpy)]  


,p_AMP (amPEPpy)↑
HydrAMP,0.41±0.13
hyformer,0.39±0.11
Random,0.39±0.12
